In [47]:
import chromatose as ct

In [48]:
import numpy as np
import pandas as pd
import scipy.optimize
import scipy.stats as st
import scipy.special

import bokeh.io
bokeh.io.output_notebook()
import panel as pn
pn.extension()

def style(p, autohide=False):
    p.title.text_font="Helvetica"
    p.title.text_font_size="16px"
    p.title.align="center"
    p.xaxis.axis_label_text_font="Helvetica"
    p.yaxis.axis_label_text_font="Helvetica"
    
    p.xaxis.axis_label_text_font_size="13px"
    p.yaxis.axis_label_text_font_size="13px"
    p.background_fill_alpha = 0
    if autohide: p.toolbar.autohide=True
    return p

Loading BokehJS ...

In [49]:
# FIXED
k_on = 1        # first dissociation

Kd_agon = 1       
Kd_endo = 10   # endogenous more likely to dissociate 10-fold

k_off_agon = Kd_agon
k_off_endo = Kd_endo

In [50]:
# TUNABLE
n = 10           # number of phosphorylation steps

k_p = 1e1       # phosphorylation  should be like 1e6???
k_1_agon = 1    # fall off agonist
k_1_endo = 0.1  # fall off endogenous

T = 1
M_agon = 1
M_endo = 1e2

In [51]:
# DEPENDENT
alpha_agon = 1/(1 + k_1_agon/k_p)
alpha_endo = 1/(1 + k_1_endo/k_p)
kappa_agon = (1-alpha_agon)*(Kd_agon + k_p/k_on)
kappa_endo = (1-alpha_endo)*(Kd_endo + k_p/k_on)

In [52]:
# steady state
Cn_agon = (alpha_agon**n)/(1-alpha_agon) * M_agon*T/(Kd_agon + k_p/k_on)
Cn_endo = (alpha_endo**n)/(1-alpha_endo) * M_endo*T/(Kd_endo + k_p/k_on)

Ctot_agon = M_agon*T/(Kd_agon + k_p/k_on) / (1-alpha_agon)
Ctot_endo = M_endo*T/(Kd_endo + k_p/k_on) / (1-alpha_endo)

In [53]:
print(Cn_agon, Cn_endo)

0.38554328942953153 457.1699121199561


In [54]:
print(Cn_agon/Ctot_agon, Cn_endo/Ctot_endo)

0.38554328942953164 0.9052869546929833


In [55]:
print(Cn_agon/(Ctot_agon+M_agon), Cn_endo/(Ctot_endo+M_endo))

0.19277164471476582 0.7556527473057132


In [56]:
Cn_agon / (M_agon + Ctot_agon)

0.19277164471476582

In [57]:
red = "#ce554e"
green = "#2ea58e"
blue = "#4d53b5"


log_k_p_slider = pn.widgets.FloatSlider(
    name="log kp", start=0, end=2, step=0.01, value=1, width=150)
log_k_1_agon_slider = pn.widgets.FloatSlider(
    name="agon log k1", start=-3, end=1, step=0.01, value=0, width=150)
log_k_1_endo_slider = pn.widgets.FloatSlider(
    name="endo log k1", start=-3, end=1, step=0.01, value=-1, width=150)
log_M_agon_slider = pn.widgets.FloatSlider(
    name="agon log Mo", start=0, end=1, step=0.01, value=0, width=150)
log_M_endo_slider = pn.widgets.FloatSlider(
    name="endo log Mo", start=1, end=5, step=0.01, value=2, width=150)

@pn.depends(log_k_p_slider.param.value, 
            log_k_1_agon_slider.param.value, 
            log_k_1_endo_slider.param.value, 
            log_M_agon_slider.param.value, 
            log_M_endo_slider.param.value)
def fm_plotter(log_k_p, log_k_1_agon, log_k_1_endo, log_M_agon, log_M_endo):
    k_p = np.power(10.0, log_k_p)
    k_1_agon = np.power(10.0, log_k_1_agon)
    k_1_endo = np.power(10.0, log_k_1_endo)        
    M_agon = np.power(10.0, log_M_agon)
    M_endo = np.power(10.0, log_M_endo) 
    
    alpha_agon = 1/(1 + k_1_agon/k_p)
    alpha_endo = 1/(1 + k_1_endo/k_p)
    kappa_agon = (1-alpha_agon)*(Kd_agon + k_p/k_on)
    kappa_endo = (1-alpha_endo)*(Kd_endo + k_p/k_on)
    
    # steady state
    n_range = np.arange(1, 100)
    fractions = np.empty((2, len(n_range)))

    for n in n_range:
        Cn_agon = (alpha_agon**n)/(1-alpha_agon) * M_agon*T/(Kd_agon + k_p/k_on)
        Cn_endo = (alpha_endo**n)/(1-alpha_endo) * M_endo*T/(Kd_endo + k_p/k_on)

        Ctot_agon = M_agon*T/(Kd_agon + k_p/k_on) / (1-alpha_agon)
        Ctot_endo = M_endo*T/(Kd_endo + k_p/k_on) / (1-alpha_endo)

        fm_agon = Cn_agon/(Ctot_agon+M_agon) 
        fm_endo = Cn_endo/(Ctot_endo+M_endo)

        fractions[0, n-1] = fm_agon
        fractions[1, n-1] = fm_endo  
        
    p = bokeh.plotting.figure(height=400, width=400, title="𝑓𝑀 vs. 𝑛", 
                              x_axis_label="n", y_axis_label="fraction M activated", 
                              y_range=(-0.1, 1.3))
    q = bokeh.plotting.figure(height=400, width=400, title="ratio agon:endo 𝑓𝑀 vs. 𝑛", 
                              x_axis_label="n", y_axis_label="ratio of fraction",
                              y_range=(0,4)
                             )

    p.circle(n_range, fractions[0], legend_label="agonist", color=red, fill_alpha=0.1, line_width=2.5, size=2)
    p.line(n_range, fractions[0], legend_label="agonist", line_color=red, line_width=1)
    
    p.circle(n_range, fractions[1], legend_label="endogenous", color=green, fill_alpha=0.1, line_width=2.5, size=2)
    p.line(n_range, fractions[1], legend_label="endogenous", line_color=green, line_width=1)
    
    q.circle(n_range, fractions[0]/fractions[1], color=blue) 
    q.line(n_range, fractions[0]/fractions[1], line_color=blue, line_width=1)
    
    if max(fractions[0]/fractions[1]) > 4:
        q.y_range=(0, max(fractions[0]/fractions[1]))
        
    return pn.Row(style(p), style(q))


lay_widgets = pn.Row(pn.Column(log_M_agon_slider, log_M_endo_slider), 
                     pn.Column(log_k_1_agon_slider, log_k_1_endo_slider), 
                     pn.Column(log_k_p_slider, align="center"), align="center")
dashboard = pn.Column(lay_widgets, fm_plotter)
dashboard

Column
    [0] Row(align='center')
        [0] Column
            [0] FloatSlider(name='agon log Mo', step=0.01, width=150)
            [1] FloatSlider(end=5, name='endo log Mo', start=1, step=0.01, value=2, width=150)
        [1] Column
            [0] FloatSlider(name='agon log k1', start=-3, step=0.01, width=150)
            [1] FloatSlider(name='endo log k1', start=-3, step=0.01, value=-1, width=150)
        [2] Column(align='center')
            [0] FloatSlider(end=2, name='log kp', step=0.01, value=1, width=150)
    [1] ParamFunction(function)

These plots are kind of bland, but very informative. I've created a static graphic below to make my observations more clear. Lose sensitivity as n goes up, and a greater fraction of 

System gets more selective.. with more n! This is fantastic

In [135]:
palette_red = [
     '#8b0000',
     '#960e0d',
     '#a11c1a',
     '#ac2a27',
     '#b73834',
     '#c24a48',
     '#cd5c5c',
     '#ce6c70',
     '#d07d85',
     '#d28d99',
     '#d49eae',
     '#d6aec3',
     '#d8bfd8',
     '#dec9de',
     '#e5d4e5'
    ]
palette_blue = [
     '#2a1c54',
     '#382670',
     '#46308d',
     '#483a99',
     '#4a45a5',
     '#4b4cad',
     '#4d53b5',
     '#5e64bc',
     '#7075c3',
     '#8286ca',
     '#9497d2',
     '#a5a8d9',
     '#b7bae1',
     '#c9cbe8',
     '#dbdcf0']
palette_red = ['#a10e47',
 '#b5134a',
 '#c7174e',
 '#d82454',
 '#e9355b',
 '#f34967',
 '#f85f75',
 '#fb7385',
 '#fc8698',
 '#fc99ac',
 '#fcadc2',
 '#fcbfd9',
 '#fdd1f0',
 '#fee1ff',
 '#ffebff']

M_agon = np.power(10.0, 0)
M_endo = np.power(10.0, 5) 

k_p = np.power(10.0, 1)
k_1_endo = np.power(10.0, -1)
k_1_agon_range = np.logspace(-2, 1, 20) # agonist fall-off rate is different

p = bokeh.plotting.figure(height=375, width=500, title="Varying agonist k_1: 𝑓𝑀 vs. 𝑛", 
                          x_axis_label="n", y_axis_label="fraction M activated", 
                          y_range=(-0.1, 1.3))
q = bokeh.plotting.figure(height=375, width=500, title="ratio agon:endo 𝑓𝑀 vs. 𝑛", 
                          x_axis_label="n", y_axis_label="ratio of fraction",
                          y_range=(-0.2,3)
                         )
for k_1_agon, _red, _blue in zip(k_1_agon_range, palette_red, palette_blue):
    alpha_agon = 1/(1 + k_1_agon/k_p)
    alpha_endo = 1/(1 + k_1_endo/k_p)
    kappa_agon = (1-alpha_agon)*(Kd_agon + k_p/k_on)
    kappa_endo = (1-alpha_endo)*(Kd_endo + k_p/k_on)

    n_range = np.arange(1, 100)
    fractions = np.empty((2, len(n_range)))

    for n in n_range:
        Cn_agon = (alpha_agon**n)/(1-alpha_agon) * M_agon*T/(Kd_agon + k_p/k_on)
        Cn_endo = (alpha_endo**n)/(1-alpha_endo) * M_endo*T/(Kd_endo + k_p/k_on)

        Ctot_agon = M_agon*T/(Kd_agon + k_p/k_on) / (1-alpha_agon)
        Ctot_endo = M_endo*T/(Kd_endo + k_p/k_on) / (1-alpha_endo)

        fm_agon = Cn_agon/(Ctot_agon+M_agon) 
        fm_endo = Cn_endo/(Ctot_endo+M_endo)

        fractions[0, n-1] = fm_agon
        fractions[1, n-1] = fm_endo  
        
    p.circle(n_range, fractions[0], color=_red, fill_alpha=0.1, line_width=1, size=1)
    p.line(n_range, fractions[0], line_color=_red, line_width=2.6)
    
    p.circle(n_range, fractions[1], legend_label="endogenous", 
             color=green, fill_alpha=0.1, line_color="white", line_width=1, size=5)
    p.line(n_range, fractions[1], legend_label="endogenous", 
             line_color=green, line_width=4)
    
    q.circle(n_range, fractions[0]/fractions[1], color=_blue, size=1) 
    q.line(n_range, fractions[0]/fractions[1], line_color=_blue, line_width=2.6)

for _palette, _plot, _direction in zip([palette_red, palette_blue],[p, q], ["left", "right"]):
    items = [(f"k_1={np.round(k_1_agon,2)}", [p.circle(color=f"{_color}"), ]) 
         for k_1_agon, _color in zip(k_1_agon_range[::2], _palette[::2])]
    legend = bokeh.models.Legend(items=items, location="center")
    _plot.add_layout(legend, _direction)
    
layout = bokeh.layouts.layout([[style(p), style(q)]])
bokeh.io.show(layout)

Increasing the number of steps can increase and decrease selectivity depending on k_1, this makes sense since high k_1 will make it fall off too quickly
when  k1 is  higher, selectivity reverses its trend 

In [19]:
k_1_agon_range = np.logspace(-2, 1, 20) # agonist fall-off rate is different

In [20]:
# .... DERIVATIVES .... 
def derivs(concs, t, k_on, k_off, k_p, k_1, k_n):
    M, T, C0, C1, C2, C3, C4, C5, C6, C7, C8, C9, Cn = concs
    
    Ci_sum = C1 + C2 + C3 + C4 + C5 + C6 + C7 + C8 + C9
    deriv_Cm = -k_on*M*T + k_off*C0 + k_1*Ci_sum + k_n*Cn
    deriv_Ct = deriv_Cm
    deriv_C0 = k_on*M*T - k_off*C0 - k_p*C0
    deriv_C1 = k_p*C0 - (k_p+k_1)*C1
    deriv_C2 = k_p*C1 - (k_p+k_1)*C2
    deriv_C3 = k_p*C2 - (k_p+k_1)*C3
    deriv_C4 = k_p*C3 - (k_p+k_1)*C4
    deriv_C5 = k_p*C4 - (k_p+k_1)*C5
    deriv_C6 = k_p*C5 - (k_p+k_1)*C6
    deriv_C7 = k_p*C6 - (k_p+k_1)*C7
    deriv_C8 = k_p*C7 - (k_p+k_1)*C8
    deriv_C9 = k_p*C8 - (k_p+k_1)*C9
    deriv_Cn = k_p*C9 - k_n*Cn
    
    return np.array(
        [deriv_Cm,
         deriv_Ct,
         deriv_C0,
         deriv_C1,
         deriv_C2,
         deriv_C3,
         deriv_C4,
         deriv_C5,
         deriv_C6,
         deriv_C7,
         deriv_C8,
         deriv_C9, 
         deriv_Cn,   
        ])

In [27]:
k_on = 10
k_off_agon = 0.1
k_off_endo = 1.0

To = 1
Mo_endo = 1e5

k_1_agon = 1e-2

In [41]:
# agonist widgets
log_Mo_agon_slider = pn.widgets.FloatSlider(name="log Mo", start=-2, end=2, step=0.01, value=0, width=150)
log_k1_agon_slider = pn.widgets.FloatSlider(name="log k1", start=-5, end=1, step=0.01, value=-2, width=150)

@pn.depends(log_Mo_agon_slider.param.value, log_k1_agon_slider.param.value)
def k1_sensitivity_explorer(log_Mo, log_k1):
    t = np.linspace(0, 5, 1000)
    Mo_agon = np.power(10.0, log_Mo)
    k_1_agon = np.power(10.0, log_k1)
    
    concs_init_agon = np.array([Mo_agon, To, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
    args_agon=(k_on, k_off_agon, k_p, k_1_agon, k_1_agon)
    _trajectories_agon = scipy.integrate.odeint(derivs, concs_init_agon, t, args=args_agon)
    M_agon, T, C0, C1, C2, C3, C4, C5, C6, C7, C8, C9, Cn_agon = _trajectories_agon.T

    concs_init_endo = np.array([Mo_endo, To, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
    args_endo=(k_on, k_off_endo, k_p, k_1_endo, k_1_endo)
    _trajectories_endo = scipy.integrate.odeint(derivs, concs_init_endo, t, args=args_endo)
    M_endo, T, C0, C1, C2, C3, C4, C5, C6, C7, C8, C9, Cn_endo = _trajectories_endo.T

    p = bokeh.plotting.figure(height=300, width=400, title="Cn Responses", 
                              x_axis_label="time", y_axis_label="[Cn]")
    p.circle(t, Cn_agon, color=red)
    p.circle(t, Cn_endo, color=green)
    return style(p)

In [42]:
widgets = pn.Row(log_Mo_agon_slider, log_k1_agon_slider, align="center")
dashboard = pn.Column(widgets, k1_sensitivity_explorer)
dashboard

Column
    [0] Row(align='center')
        [0] FloatSlider(end=2, name='log Mo', start=-2, step=0.01, width=150)
        [1] FloatSlider(name='log k1', start=-5, step=0.01, value=-2, width=150)
    [1] ParamFunction(function)